In [ ]:
# Pip install (recommended)

!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.1/562.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 25.3 MB/s eta 0:00:00


In [ ]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_pollen_data/images/train/"
train_path_label = "./yolo_pollen_data/labels/train/"
val_path_img = "./yolo_pollen_data/images/val/"
val_path_label = "./yolo_pollen_data/labels/val/"
test_path = "./yolo_pollen_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8_PollenV2/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 2213 images---


  0%|          | 0/1771 [00:00<?, ?it/s]

------ Training data created with 80% split 1771 images -------


  0%|          | 0/442 [00:00<?, ?it/s]

------ Testing data created with a total of 442 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.5/78.2 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=/content/drive/MyDrive/yolov8_PollenV2/data.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8_PollenV2/test_results name=pollen 

100% 131M/131M [00:18<00:00, 7.38MB/s]
Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/yolov8_PollenV2/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8_PollenV2/test_results, name=pollen, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8_PollenV2/test_results/pollen3/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest save=True

Ultralytics YOLOv8.0.92 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68142828 parameters, 0 gradients, 257.5 GFLOPs

image 1/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115410.png: 640x640 (no detections), 81.5ms
image 2/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115455.png: 576x640 (no detections), 77.4ms
image 3/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115529.png: 640x608 (no detections), 96.8ms
image 4/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115552.png: 384x640 (no detections), 65.8ms
image 5/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115607.png: 480x640 (no detections), 76.6ms
image 6/6 /content/drive/MyDrive/yolov8_PollenV2/04-05_FirstTest/Skærmbillede 2023-05-04 115623.png: 544x640 (no detections), 91.9ms
Speed: 4.5ms preprocess, 81.7ms 

In [ ]:
!cp -r /content/runs/detect/predict/ /content/drive/MyDrive/yolov8_PollenV2/outputTest0405